# Todo List
- konstruktor
- Methoden (update, over)
- Datenstruktur Spielfeld (2D-array)
- Funktion (fieldEmpty?)
- in over() Struktur winningConditions
- class result
- class position 8pair)
- tests
- später GUI